# Study of the Asortativity in Topic

In [1]:
import os
# Verificación y ubicación (y descarga de archivos en colab)
directory_path = os.getcwd()
folder_name = os.path.basename(directory_path)
if folder_name != 'label propagation':
    try:
        os.chdir("/content/CoNet/Tp Final/analysis/label propagation")
    except FileNotFoundError:
        !git clone https://github.com/facundojgarcia02/CoNet.git
        os.chdir("/content/CoNet/Tp Final/analysis/label propagation")

In [12]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import pickle5 as pickle
from random import sample, seed
from utils.metrics import accuracy, balanced_accuracy

from matplotlib import rcParams
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
rcParams['font.family'] = 'serif'
rcParams['mathtext.fontset'] = 'cm'
rcParams['figure.dpi'] = 100
rcParams['legend.fontsize'] = 10
rcParams['axes.labelsize'] = 'large'

seed(42)

def convert_str_to_list(s):
    try:
        if np.isnan(s):
            return None
    except TypeError:
        return list(set(s.replace("['", "").replace("']", "").split("', '")))

In [3]:
# Load Graph.
G = nx.read_gexf("../../state_files/PyPi Network V4.gexf")
# Keep only giant component.
gc_nodes = sorted(nx.connected_components(G.to_undirected()), key = lambda x: len(x), reverse=True)[0]
not_gc_nodes = set(G.nodes()) - gc_nodes
G.remove_nodes_from(not_gc_nodes)

In [4]:
# # Load labels.
# with open("../../state_files/PyPi Dataframe V4.pickle", "rb") as f:
#     df = pickle.load(f)
# labels = df["Topic"].dropna().to_dict()
# # Remove 'Other' topic.
# for n, l in labels.items():
#     if 'Other/Nonlisted Topic' in l:
#         l.remove('Other/Nonlisted Topic')
# Load labels.
topics_series = pd.read_csv("../../state_files/TopicsDataframe V4.tsv", sep="\t", index_col=0,)["Topic"]
topics_series
labels = topics_series.dropna().apply(convert_str_to_list).to_dict()
# Remove 'Other' topic.
for n, l in labels.items():
    if 'Other/Nonlisted Topic' in l:
        l.remove('Other/Nonlisted Topic')

In [5]:
# Filter nodes with no topics.
labels = dict(filter(lambda x: len(x[1]) > 0, labels.items()))
# Keep nodes from giant component.
labels = dict(filter(lambda x: x[0] in gc_nodes, labels.items()))

In [6]:
# Split train and test set.
test_size = 0.1

test_nodes = sample(list(labels.keys()), int(test_size*len(labels.keys())))
train_nodes = list(filter(lambda x: x not in test_nodes, labels.keys()))

train_labels = dict(filter(lambda x: x[0] in train_nodes, labels.items()))
test_labels = dict(filter(lambda x: x[0] in test_nodes, labels.items()))

In [7]:
label_dist = []
for lib, topic_set in train_labels.items():
    label_dist += list(topic_set)
pred_list = sample(label_dist, len(test_labels))
pred_labels = {lib: [pred] for lib, pred in zip(test_labels, pred_list)}
# pred_labels

In [10]:
accuracy(test_labels, pred_labels)

Accuracy: 29%


0.28606923968858705

In [13]:
balanced_accuracy(test_labels, pred_labels)

(0.0,
 {'Scientific/Engineering': 1639,
  'Security': 167,
  'Terminals': 55,
  'Home Automation': 58,
  'System': 503,
  'Text Editors': 19,
  'Printing': 9,
  'Software Development': 3225,
  'Communications': 150,
  'Desktop Environment': 21,
  'Internet': 841,
  'Religion': 3,
  'Multimedia': 202,
  'Education': 100,
  'Sociology': 7,
  'Text Processing': 268,
  'Games/Entertainment': 73,
  'Documentation': 78,
  'Utilities': 950,
  'Artistic Software': 15,
  'Adaptive Technologies': 3,
  'Office/Business': 157,
  'Database': 152})